Finite difference method for boundary value problem (Book 2, p. 10, ex. 1.5).

# Common parts

In [18]:
from numpy import exp, arange

We have to solve this task:

$-\frac{1}{x-3} u^{\prime \prime}+\left(1+\frac{x}{2}\right) u^{\prime}+e^{x / 2} u=2-x, u(-1)=u(1)=0$

In [22]:
def p(x):
    return 1/(x-3)

def q(x):
    return 1 + x/2

def r(x):
    return exp(x/2)

def f(x):
    return 2 - x

a = -1
b = 1
alpha = 0
beta = 0

# Differential equation approximation 

In [23]:
def prepare_coefficents(p, q, r, f, h, x):
    """Preparing equation coefficents.
    
    Suppose we given a boundary value problem 
    with bounds (a, b) and functions p, q, r, f. 
    We want to find values of  desired function 
    on n splits on (a, b). So we have step 
    of h=(b-a)/n. 
    Later we swap our differential formulas with
    their approximate values and grouping 
    coeficents near y_i values.
    
    Args:
        p, q, r, f (fun(x)): boundary value functions.
        h (float): step of points we want to find values in.
        x (float): point to calculate coefficents in.
    
    Returns:
        A, B, C, G (fun(x)): coefficents near y_i values.
    
    """
    
    A = -(p(x) / h**2) - (q(x) / 2*h)
    B = -(2*p(x) / h**2) - r(x)
    C = -(p(x) / h**2) + (q(x) / 2*h)
    G = f(x)
    
    return A, B, C, G

In [28]:
def solve_equation(p, q, r, f, a, b, alpha, beta, n):
    """Solve differential equation with boundaries.
    
    Args:
        p, q, r, f (fun(x)): boundary value functions.
        a, b (float): Bounds.
        alpha, beta (float): Bound values.
        n (int): Number of splits.
    
    Returns:
        Y (array-like<float>): Values of desired function
                              in (n + 1) points on (a, b)
                              with step h=(b-a)/n.
    """
    
    h = (b - a) / n
    points = arange(a, b + h, h)
    
    return h, len(points)
    

In [15]:
_, _, _, G = prepare_coefficents(p, q, r, f, (b-a)/10, 0.5)

In [16]:
G

1.5

In [29]:
solve_equation(p, q, r, f, a, b, alpha, beta, 10)

(0.2, 11)